In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../scripts'))

In [3]:
# Load your DataFrame with the correct delimiter
df = pd.read_csv("../Data/cleaned.csv")
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,premium_per_claim
0,145249,12827.0,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,1.0,21.929825
1,145249,12827.0,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,1.0,21.929825
2,145249,12827.0,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,1.0,0.000000
3,145255,12827.0,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,1.0,512.848070
4,145255,12827.0,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,1.0,0.000000


In [4]:
df.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'SumInsured', 'TermFrequency',
       'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory',
       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType', 'TotalPremium', 'TotalClaims',
       'premium_per_claim'],
      dtype='object')

In [5]:
from ab_testing import ABHypothesisTesting

In [6]:
# Initialize the hypothesis testing class
hypothesis_testing = ABHypothesisTesting(df)

In [7]:
# Test if there are risk differences across provinces
province_test_result = hypothesis_testing.test_risk_across_provinces()
province_test_result

{'Test': 'ANOVA',
 'Null Hypothesis': 'No risk differences across provinces',
 'F-Statistic': np.float64(3.921779592906316),
 'p-Value': np.float64(0.00012053268919307433),
 'Reject Null': np.True_}

In [8]:
# Check the number of observations per group
print(df['PostalCode'].value_counts())

PostalCode
2000.0    125151
122.0      27206
7784.0     20461
299.0      18772
7405.0     14659
           ...  
356.0          8
340.0          8
7350.0         1
7280.0         1
7760.0         1
Name: count, Length: 681, dtype: int64


In [9]:
# Remove postal codes with fewer than 5 observations (you can adjust this threshold as needed)
df_filtered = df[df.groupby('PostalCode')['PostalCode'].transform('count') > 5]

# Now you can re-run your ANOVA or Kruskal-Wallis test

In [10]:
# Test if there are risk differences between postal codes
postalcode_test_result = hypothesis_testing.test_risk_between_PostalCode()
postalcode_test_result

{'Test': 'ANOVA',
 'Null Hypothesis': 'No risk differences between postal codes',
 'F-Statistic': np.float64(1.0220078560484607),
 'p-Value': np.float64(0.33699273849573924),
 'Reject Null': np.False_}

In [11]:
# Test if there are significant margin differences between zip codes
margin_test_result = hypothesis_testing.test_margin_difference_between_PostalCode()
margin_test_result

{'Test': 'ANOVA',
 'Null Hypothesis': 'No significant margin differences between postal codes',
 'F-Statistic': np.float64(0.9368688522740898),
 'p-Value': np.float64(0.8791631359151926),
 'Reject Null': np.False_}

In [12]:
kpi = df["TotalClaims"] / df["TotalPremium"]
df["RiskRatio"] = kpi


In [13]:
group_a = df[df["TotalPremium"] == 0]
group_b = df[df["TotalPremium"] == 1]


In [15]:
from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(group_a["TotalPremium"], group_b["TotalPremium"])
print(f"P-value: {p_value}")


P-value: nan


C:\Users\hp\AppData\Local\Temp\ipykernel_4344\2630482462.py:3: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_stat, p_value = ttest_ind(group_a["TotalPremium"], group_b["TotalPremium"])
